In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone https://github.com/piantado/LOTlib3
else:
    print("Please download LOTlib3 in this folder!")
    
import LOTlib3

Please download LOTlib3 in this folder!


ModuleNotFoundError: No module named 'LOTlib3'

# Grammar

In [5]:
from LOTlib3.Grammar import Grammar
grammar = Grammar()

grammar.add_rule('START', '', ['QUANT'], 1.0)

# Very simple -- one allowed and required quantifier
grammar.add_rule('QUANT', 'exists_', ['FUNCTION', 'SET'], 1.00)
grammar.add_rule('QUANT', 'forall_', ['FUNCTION', 'SET'], 1.00)

# The thing we are a function of
grammar.add_rule('SET', 'S', None, 1.0)

# And allow us to create a new kind of function
grammar.add_rule('FUNCTION', 'lambda', ['BOOL'], 1.0, bv_type='OBJECT')

# Logical connectives
grammar.add_rule('BOOL', 'and_', ['BOOL', 'BOOL'], 1.0)
grammar.add_rule('BOOL', 'or_', ['BOOL', 'BOOL'], 1.0)
grammar.add_rule('BOOL', 'not_', ['BOOL'], 1.0)

# non-terminal arguments get passed as normal python arguments
grammar.add_rule('BOOL', 'is_color_',  ['OBJECT', '\'red\''],   5.00) # --> is_color_(OBJECT, 'red') --> OBJECT.color == 'red'
grammar.add_rule('BOOL', 'is_color_',  ['OBJECT', '\'blue\''],  5.00)
grammar.add_rule('BOOL', 'is_color_',  ['OBJECT', '\'green\''], 5.00)

C:\Users\faust\Dropbox\Tubingen\teaching\pLoT\pLoT_course\LOTlib3\Miscellaneous.py:190: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and (slot is not "__doc__") and (slot is not "__module__"):
C:\Users\faust\Dropbox\Tubingen\teaching\pLoT\pLoT_course\LOTlib3\Miscellaneous.py:190: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and (slot is not "__doc__") and (slot is not "__module__"):
C:\Users\faust\Dropbox\Tubingen\teaching\pLoT\pLoT_course\LOTlib3\FunctionNode.py:833: SyntaxWarning: "is" with a literal. Did you mean "=="?
  assert name is 'lambda'


BOOL -> is_color_['OBJECT', "'green'"]	w/ p=5.0

# Hypothesis

In [6]:
from LOTlib3.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib3.Hypotheses.Likelihoods.BinaryLikelihood import BinaryLikelihood

class MyHypothesis(BinaryLikelihood, LOTHypothesis):
    def __init__(self, grammar=grammar, **kwargs):
        LOTHypothesis.__init__(self, grammar=grammar, display='lambda S: %s', **kwargs)

# Data

In [6]:
from LOTlib3.DataAndObjects import FunctionData, Obj # for nicely managing data

# Make up some data -- here just one set containing {red, red, green} colors that is mapped to True
def make_data(n=1):
    return [ 
        FunctionData(
            input=[ {Obj(color='red'), Obj(color='red'), Obj(color='green')} ], 
            output=True, 
            alpha=0.99
        ) 
    ]*n

C:\Users\faust\Dropbox\Tubingen\teaching\pLoT\pLoT_course\LOTlib3\Primitives\Semantics.py:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if    x is "undefT": return True
C:\Users\faust\Dropbox\Tubingen\teaching\pLoT\pLoT_course\LOTlib3\Primitives\Semantics.py:32: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif  x is "undefF": return False


# Main

In [8]:
from LOTlib3 import break_ctrlc
from LOTlib3.Miscellaneous import qq
from LOTlib3.TopN import TopN
from LOTlib3.Samplers.MetropolisHastings import MetropolisHastingsSampler

h0   = MyHypothesis()
data = make_data()
top  = TopN(N=10)
thin = 100

for i, h in enumerate(break_ctrlc(MetropolisHastingsSampler(h0, data))):

    top << h

    if i % thin == 0:
        print("#", i, h.posterior_score, h.prior, h.likelihood, qq(h))


# 0 -7.272398392570045 -1.9740810260220094 -5.298317366548035 "lambda S: forall_(lambda y2: is_color_(y2, 'green'), S)"
# 100 -1.9790935678455537 -1.9740810260220094 -0.005012541823544286 "lambda S: exists_(lambda y2: is_color_(y2, 'green'), S)"
# 200 -1.9790935678455537 -1.9740810260220094 -0.005012541823544286 "lambda S: exists_(lambda y2: is_color_(y2, 'green'), S)"
# 300 -4.869465325741718 -4.864452783918174 -0.005012541823544286 "lambda S: exists_(lambda y2: not_(is_color_(y2, 'red')), S)"
# 400 -1.9790935678455537 -1.9740810260220094 -0.005012541823544286 "lambda S: exists_(lambda y2: is_color_(y2, 'green'), S)"
# 500 -1.9790935678455537 -1.9740810260220094 -0.005012541823544286 "lambda S: exists_(lambda y2: is_color_(y2, 'red'), S)"
# 600 -1.9790935678455537 -1.9740810260220094 -0.005012541823544286 "lambda S: exists_(lambda y2: is_color_(y2, 'red'), S)"
# 700 -1.9790935678455537 -1.9740810260220094 -0.005012541823544286 "lambda S: exists_(lambda y2: is_color_(y2, 'red'), S)"
# 

KeyboardInterrupt: 

In [10]:
for h in top:
    print(h.posterior_score, h.prior, h.likelihood, qq(h))

-6.150399171203782 -6.145386629380238 -0.005012541823544286 "lambda S: exists_(lambda y2: or_(is_color_(y2, 'green'), is_color_(y2, 'blue')), S)"
-6.150399171203782 -6.145386629380238 -0.005012541823544286 "lambda S: exists_(lambda y2: or_(is_color_(y2, 'red'), is_color_(y2, 'green')), S)"
-6.150399171203782 -6.145386629380238 -0.005012541823544286 "lambda S: exists_(lambda y2: or_(is_color_(y2, 'green'), is_color_(y2, 'green')), S)"
-6.150399171203782 -6.145386629380238 -0.005012541823544286 "lambda S: exists_(lambda y2: and_(is_color_(y2, 'green'), is_color_(y2, 'green')), S)"
-4.869465325741718 -4.864452783918174 -0.005012541823544286 "lambda S: exists_(lambda y2: not_(is_color_(y2, 'blue')), S)"
-4.869465325741718 -4.864452783918174 -0.005012541823544286 "lambda S: exists_(lambda y2: not_(is_color_(y2, 'red')), S)"
-4.869465325741718 -4.864452783918174 -0.005012541823544286 "lambda S: exists_(lambda y2: not_(is_color_(y2, 'green')), S)"
-4.869465325741718 -4.864452783918174 -0.0050